In [1]:
!pip install tensorflow-gpu
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 9.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import keras_tuner
from kerastuner.tuners import RandomSearch

<ipython-input-2-1ac5c712ab08>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
# Download  dataset
dataset, info = tfds.load('horses_or_humans', with_info=True, as_supervised=True)

# Split the dataset into training and test sets
train_dataset = dataset['train']
test_dataset = dataset['test']

print(info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteJMBAWS/horses_or_humans-train.tfrecord*..…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteJMBAWS/horses_or_humans-test.tfrecord*...…

Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='horses_or_humans',
    full_name='horses_or_humans/3.0.0',
    description="""
    A large set of images of horses and humans.
    """,
    homepage='http://laurencemoroney.com/horses-or-humans-dataset',
    data_path='/root/tensorflow_datasets/horses_or_humans/3.0.0',
    file_format=tfrecord,
    download_size=153.59 MiB,
    dataset_size=153.53 MiB,
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=256, num_shards=1>,
        'train': <SplitInfo num_examples=1027, num_shards=2>,
    },
    citation="""@ONLINE {horses_or_humans,
    author = "Laurence Moroney",
    title = "Horses or 

In [4]:
batch_size = 32
epochs = 5

In [5]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    image = tf.image.resize(image, (300, 300))
    return image, label

In [6]:
# Create train and test sets

train_dataset = train_dataset.map(preprocess).batch(batch_size)
X_train = []
y_train = []

for images, labels in train_dataset.as_numpy_iterator():
    X_train.append(images)
    y_train.append(labels)

X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)


test_dataset = test_dataset.map(preprocess).batch(batch_size)
X_test = []
y_test = []

for images, labels in test_dataset.as_numpy_iterator():
    X_test.append(images)
    y_test.append(labels)

X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

In [7]:

X_train.shape, X_test.shape

((1027, 300, 300, 3), (256, 300, 300, 3))

In [8]:
# Create validation set from training set

train_val_split = int(0.9*X_train.shape[0])
print(train_val_split)

X_val, y_val = X_train[train_val_split:], y_train[train_val_split:]
X_train, y_train = X_train[:train_val_split], y_train[:train_val_split]
     

924


In [9]:
X_train.shape, X_val.shape, X_test.shape

((924, 300, 300, 3), (103, 300, 300, 3), (256, 300, 300, 3))

In [10]:
model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(300,300,3)),
            tf.keras.layers.MaxPooling2D((2,2)),
            tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2,2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(20, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
            ])

# Train model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/5
29/29 [==============================] - 16s 115ms/step - loss: 0.5808 - accuracy: 0.7327 - val_loss: 0.1471 - val_accuracy: 0.9223
Epoch 2/5
29/29 [==============================] - 2s 75ms/step - loss: 0.0677 - accuracy: 0.9816 - val_loss: 0.0279 - val_accuracy: 1.0000
Epoch 3/5
29/29 [==============================] - 2s 66ms/step - loss: 0.0199 - accuracy: 0.9935 - val_loss: 0.0409 - val_accuracy: 0.9806
Epoch 4/5
29/29 [==============================] - 2s 62ms/step - loss: 0.0159 - accuracy: 0.9968 - val_loss: 0.0112 - val_accuracy: 1.0000
Epoch 5/5
8/8 [==============================] - 0s 30ms/step - loss: 2.4458 - accuracy: 0.7188
Test accuracy: 0.71875


##Hyperparameter tuning

In [11]:
def create_model(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(hp.Int('filters1', 32, 96, step=32), (3,3), activation='relu', input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.MaxPooling2D((2,2)))
  model.add(tf.keras.layers.Conv2D(hp.Int('filters2', 32, 96, step=32), (3,3), activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D((2,2)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(hp.Int('units1', 10, 50, step=32), activation='relu'))
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  # Train    
  model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])), loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [12]:
# Define tuner

tuner = RandomSearch(
    create_model,
    objective='val_accuracy',
    max_trials=epochs,
    executions_per_trial=2,
    directory='keras_tuner',
)

In [13]:
# Search for the best hyperparameters

tuner.search(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

Trial 5 Complete [00h 01m 03s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 04m 34s


In [14]:
# Print best hyperparameters

best_hyperparams = tuner.get_best_hyperparameters()[0]
print('Best hyperparameters:')
print(f'learning_rate = {best_hyperparams.get("learning_rate")}')
print(f'filters1 = {best_hyperparams.get("filters1")}')
print(f'filters2 = {best_hyperparams.get("filters2")}')
print(f'units1 = {best_hyperparams.get("units1")}')

Best hyperparameters:
learning_rate = 0.001
filters1 = 64
filters2 = 64
units1 = 42


In [15]:
# Create a new model with the best hyperparameters

model = tuner.hypermodel.build(best_hyperparams)

model.compile(optimizer=tf.keras.optimizers.Adam(best_hyperparams.get('learning_rate')), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with the best hyperparameters
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

# Evaluate the model on the test set with the best hyperparameters
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy with hyperparameter tuning:', test_acc)

Epoch 1/5
29/29 [==============================] - 5s 129ms/step - loss: 1.5031 - accuracy: 0.7435 - val_loss: 0.1821 - val_accuracy: 0.9417
Epoch 2/5
29/29 [==============================] - 4s 124ms/step - loss: 0.0905 - accuracy: 0.9686 - val_loss: 0.0346 - val_accuracy: 1.0000
Epoch 3/5
29/29 [==============================] - 3s 110ms/step - loss: 0.0156 - accuracy: 0.9968 - val_loss: 0.0085 - val_accuracy: 1.0000
Epoch 4/5
29/29 [==============================] - 3s 110ms/step - loss: 0.0124 - accuracy: 0.9957 - val_loss: 0.0037 - val_accuracy: 1.0000
Epoch 5/5
8/8 [==============================] - 0s 45ms/step - loss: 2.7449 - accuracy: 0.7148
Test accuracy with hyperparameter tuning: 0.71484375


##Data augmentation + Hyperparameter tuning

In [16]:
# Data augmentation pipeline

data_augmentation = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
                                         tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
                                         tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)])
     

In [17]:
def create_model_(hp):
    model = tf.keras.Sequential([
        data_augmentation,
        tf.keras.layers.Conv2D(hp.Choice('filters1', values=[32, 64]), kernel_size=3, activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.Conv2D(hp.Choice('filters2', values=[32, 64]), kernel_size=3, activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.2),
        tf.keras. layers.Flatten(),
        tf.keras.layers.Dense(hp.Choice('units1', values=[20, 50, 100]), activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model
     

In [18]:
# Define the hyperparameter search space

tuner_search = RandomSearch(
    create_model_,
    objective='val_accuracy',
    max_trials=epochs,
    executions_per_trial=2,
    directory='tuner_dir',
)
 

In [19]:
# Perform the hyperparameter search

tuner_search.search(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

Trial 5 Complete [00h 02m 10s]
val_accuracy: 0.8155339956283569

Best val_accuracy So Far: 0.8980582356452942
Total elapsed time: 00h 11m 52s


In [20]:
best_hp = tuner_search.get_best_hyperparameters(num_trials=1)[0]
print(f'Best hyperparameters:\n{best_hp.values}\n')

Best hyperparameters:
{'filters1': 32, 'filters2': 64, 'units1': 20, 'learning_rate': 0.0001}



In [21]:
model = tuner_search.hypermodel.build(best_hp)
     

In [22]:
model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/5
29/29 [==============================] - 11s 318ms/step - loss: 1.2343 - accuracy: 0.4784 - val_loss: 0.6931 - val_accuracy: 0.5049
Epoch 2/5
29/29 [==============================] - 9s 303ms/step - loss: 0.6931 - accuracy: 0.5141 - val_loss: 0.6931 - val_accuracy: 0.5049
Epoch 3/5
29/29 [==============================] - 9s 303ms/step - loss: 0.6931 - accuracy: 0.5141 - val_loss: 0.6931 - val_accuracy: 0.5049
Epoch 4/5
29/29 [==============================] - 9s 298ms/step - loss: 0.6931 - accuracy: 0.5141 - val_loss: 0.6931 - val_accuracy: 0.5049
Epoch 5/5
29/29 [==============================] - 9s 301ms/step - loss: 0.6931 - accuracy: 0.5141 - val_loss: 0.6931 - val_accuracy: 0.5049


In [23]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy with hyperparameter tuning and data augmentation: {test_acc}')

8/8 [==============================] - 0s 58ms/step - loss: 0.6931 - accuracy: 0.5000
Test accuracy with hyperparameter tuning and data augmentation: 0.5
